- https://bybit-exchange.github.io/docs/v5/intro

- v5/market/	Candlestick, orderbook, ticker, platform transaction data, underlying financial rules, risk control rules
    - /v5/market/kline
    - /v5/market/tickers
- v5/order/	Order management
- v5/position/	Position management
- v5/account/	Single account operations only – unified funding account, rates, etc.
- v5/asset/	Operations across multiple accounts – asset management, fund management, etc.


## ol import libraries

In [2]:
library(glue)
library(httr)
library(jsonlite)
library(tidyverse)

# ol get_bybit_tickers

```R
# Example usage linear
result <- get_bybit_tickers(category = "linear")
result

# Example usage spot
result <- get_bybit_tickers(category = "spot")
resul
```


In [ ]:

# Function to get Bybit tickers
get_bybit_tickers <- function(category, ticker = NULL, baseCoin = NULL, expDate = NULL) {
  # Base URL for the Bybit API
  base_url <- "https://api.bybit.com/v5/market/tickers"

  # Create a list of parameters
  params <- list(category = category)
  if (!is.null(ticker)) params$symbol <- ticker
  if (!is.null(baseCoin)) params$baseCoin <- baseCoin
  if (!is.null(expDate)) params$expDate <- expDate

  # Make the GET request
  response <- GET(url = base_url, query = params)

  # Check if the request was successful
  if (response$status_code == 200) {
    # Parse the JSON response
    data <- fromJSON(content(response, "text"), simplifyDataFrame = TRUE)
        # Assuming the data of interest is in a list within 'result.list'
    if ("result" %in% names(data) && "list" %in% names(data$result)) {
      ticker_data <- as_tibble(data$result$list) %>%
        select(
          symbol, lastPrice, price24hPcnt, volume24h, turnover24h
        ) %>%
        rename(
          ticker = symbol,
          last_price = lastPrice,
          price_24h_pct = price24hPcnt,
          volume_24h = volume24h,
          turnover_24h = turnover24h
        ) %>%
        mutate (
          across(c(last_price, price_24h_pct, volume_24h, turnover_24h), as.numeric)
        ) %>%
        arrange(desc(turnover_24h))
      return(ticker_data)
    } else {
      stop("Unexpected data structure in the response.")
    }
    return(data)
  } else {
    stop("Failed to retrieve data: ", response$status_code)
  }
}


# Get Market data

- https://bybit-exchange.github.io/docs/v5/market/kline

GET /v5/market/kline

Request Parameters (Parameter	Required	Type	Comments)
- category	true	string	Product type. spot,linear,inverse
- symbol	true	string	Symbol name
- interval	true	string	Kline interval. 1,3,5,15,30,60,120,240,360,720,D,M,W
- start	false	integer	The start timestamp (ms)
- end	false	integer	The end timestamp (ms)
- limit	false	integer	Limit for data size per page. [1, 1000]. Default: 200

## single_bybit_prices simple limited function to get candles

In [ ]:
# Function to get Bybit klines
single_bybit_prices <- function(category, symbol, interval, start, end) {
  # Base URL for the Bybit API
  base_url <- "https://api.bybit.com/v5/market/kline"

  # Create a list of parameters
  params <- list(category = category, symbol = symbol, interval = interval, limit = 1000)
  if (!is.null(start)) params$start <- as.numeric(as.POSIXct(start, tz = "UTC", format = "%Y-%m-%d")) * 1000
  if (!is.null(end)) params$end <- as.numeric(as.POSIXct(end, tz = "UTC", format = "%Y-%m-%d")) * 1000
  # if (!is.null(end)) params$end <- as.numeric(as.POSIXct(end, tz = "UTC", format = "%Y-%m-%d")+ (23 * 3600 + 59 * 60 + 59))  * 1000
  # params$limit <- 200

  # Make the GET request
  response <- GET(url = base_url, query = params)


  # Check if the request was successful
  if (response$status_code == 200) {
    # Parse the JSON response and convert to a data frame
    data <- fromJSON(content(response, "text"), flatten = TRUE)
    # print(data$result$list)

    # Assuming the data of interest is in a list within 'result.list'
    if ("result" %in% names(data) && "list" %in% names(data$result)) {
      kline_data <- data$result$list
      # Set column names
      colnames(kline_data) <- c("datetime", "open", "high", "low", "close", "volume", "turnover")
      # Convert to tibble
      kline_data <- as_tibble(kline_data)
      kline_data <- kline_data %>%
        mutate(
          datetime =  as.POSIXct(as.numeric(datetime) / 1000, origin = "1970-01-01", tz = "UTC"),
          across(c(open, high, low, close, volume, turnover), as.numeric)
        ) %>%
        arrange(datetime)
      print(length(kline_data$datetime))
      return(kline_data)
    } else {
      stop("Unexpected data structure in the response.")
    }
  } else {
    stop("Failed to retrieve data: ", response$status_code)
  }
}

# Example usage
result <- single_bybit_prices(category = "spot", start="2023-11-01", end="2023-11-13", symbol = "BTCUSDT", interval = "1")
print(result)

[1] 1000
# A tibble: 1,000 × 7
   datetime              open   high    low  close volume turnover
   <dttm>               <dbl>  <dbl>  <dbl>  <dbl>  <dbl>    <dbl>
 1 2023-11-12 07:21:00 37150  37154  37150  37154.  1.48    54895.
 2 2023-11-12 07:22:00 37154. 37161. 37154. 37154.  2.55    94824.
 3 2023-11-12 07:23:00 37154. 37154. 37146. 37146.  2.38    88548.
 4 2023-11-12 07:24:00 37146. 37146. 37132. 37135.  0.999   37110.
 5 2023-11-12 07:25:00 37135. 37135. 37129. 37134   1.53    56682.
 6 2023-11-12 07:26:00 37134  37134  37124. 37124.  0.658   24428.
 7 2023-11-12 07:27:00 37124. 37124. 37118. 37118.  0.876   32534.
 8 2023-11-12 07:28:00 37118. 37121. 37118. 37121.  1.48    55044.
 9 2023-11-12 07:29:00 37121. 37125. 37121. 37125.  1.77    65684.
10 2023-11-12 07:30:00 37125. 37125. 37112. 37112.  1.03    38144.
# ℹ 990 more rows


## single_bybit_prices2 - loop all history

In [ ]:
library(glue)

# Function to get Bybit klines
single_bybit_prices2 <- function(category, symbol, interval, start, end) {
  # Base URL for the Bybit API
  base_url <- "https://api.bybit.com/v5/market/kline"

  start_unix = as.numeric(as.POSIXct(start, tz = "UTC", format = "%Y-%m-%d")) * 1000
  end_unix = as.numeric(as.POSIXct(glue("{end} 23:59:59"), tz = "UTC", format = "%Y-%m-%d %H:%M:%S")) * 1000
  # end_unix = as.numeric(as.POSIXct(end, tz = "UTC", format = "%Y-%m-%d")+ (23 * 3600 + 59 * 60 + 59))  * 1000

  # Create a list of parameters
  params <- list(category = category, symbol = symbol, interval = interval, limit = 1000)
  params$start <- start_unix
  params$end <- end_unix

  all_data <- data.frame()

  # Loop to fetch data in chunks
  repeat {
    # Make the GET request
    response <- GET(url = base_url, query = params)

    # Check if the request was successful
    if (response$status_code == 200) {
      # Parse the JSON response and convert to a data frame
      data <- fromJSON(content(response, "text"), flatten = TRUE)
      # print(data$result$list)
      # break
      # Assuming the data of interest is in a list within 'result.list'
      if ("result" %in% names(data) && "list" %in% names(data$result)) {
        kline_data <- data$result$list

        if (length(kline_data) > 0) {
          # Set column names
          colnames(kline_data) <- c("OpenTime", "Open", "High", "Low", "Close", "Volume", "Turnover")


          # Convert to tibble
          kline_data <- as_tibble(kline_data)

          kline_data <- kline_data %>%
            mutate(
              OpenTime =  as.POSIXct(as.numeric(OpenTime) / 1000, origin = "1970-01-01", tz = "UTC"),
              across(c(Open, High, Low, Close, Volume, Turnover), as.numeric)
            ) %>%
            arrange(OpenTime)



          # Add data to all_data
          all_data <- rbind(all_data, kline_data)

          # Update start time for next request

          first_time <- min(kline_data$OpenTime)
          last_time <- max(kline_data$OpenTime)
          # start_unix <- as.numeric(last_time) * 1000 + 1
          # params$start <- start_unix
          end_unix <- as.numeric(first_time) * 1000 - 1
          params$end <- end_unix

          # Break the loop if we've fetched all data
          # print(length(kline_data$OpenTime))
          # print(as.POSIXct(as.numeric(start_unix) / 1000, origin = "1970-01-01", tz = "UTC"))
          # print(as.POSIXct(as.numeric(end_unix) / 1000, origin = "1970-01-01", tz = "UTC"))
          if (length(kline_data$OpenTime) < params$limit || start_unix > end_unix) {
            # print(start_unix > end_unix)
            # print("breaking")
            break
          }
        } else {
          # no data received
          break
        }

        # return(kline_data)
      } else {
        warning("Unexpected data structure in the response.")
        return(all_data)
      }
    } else {
      # stop("Failed to retrieve data: ", response$status_code)
      warning("Request failed. Status: ", http_status(response)$status_code, " - ", http_status(response)$reason)
      return(all_data)
    }
  }

  if (length(all_data) > 0) {
    all_data <- all_data %>%
    arrange(OpenTime)
  }


   return(all_data)
}

# Example usage
result <- single_bybit_prices2(category = "spot", start="2023-11-01", end="2023-11-02", symbol = "BTCUSDT", interval = "1")
# print(result)
head(result)
tail(result)
# resul

OpenTime,Open,High,Low,Close,Volume,Turnover
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2023-11-01 00:00:00,34641.07,34656.70,34636.11,34654.90,4.753187,164691.8
2023-11-01 00:01:00,34654.90,34665.23,34630.17,34630.17,7.723151,267626.6
2023-11-01 00:02:00,34630.17,34637.57,34623.20,34637.57,4.799330,166204.6
2023-11-01 00:03:00,34637.57,34637.57,34603.19,34609.18,7.156501,247751.9
2023-11-01 00:04:00,34609.18,34610.00,34584.26,34599.99,4.803048,166161.1
2023-11-01 00:05:00,34599.99,34605.98,34587.82,34591.64,4.470993,154688.4


OpenTime,Open,High,Low,Close,Volume,Turnover
<dttm>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2023-11-02 23:54:00,34920.00,34920.00,34919.65,34919.66,0.159987,5586.712
2023-11-02 23:55:00,34919.66,34923.45,34910.89,34923.44,5.790683,202181.264
2023-11-02 23:56:00,34923.44,34930.00,34923.44,34930.00,2.568947,89722.574
2023-11-02 23:57:00,34930.00,34935.98,34929.99,34935.97,0.844876,29512.624
2023-11-02 23:58:00,34935.97,34935.98,34929.85,34930.34,3.156807,110275.842
2023-11-02 23:59:00,34930.34,34941.12,34928.91,34940.76,2.977211,104004.877


## bybit_prices - get multiple symbols candles history

In [ ]:
bybit_prices <- function(category = "spot", symbols, interval = "60", start, end) {
  # Helper function for adding a Ticker column and arranging columns
  fun <- function(category, symbol, interval, start, end) {
    prices <- single_bybit_prices2(
      category, symbol, interval, start, end
    )
    if (length(prices) > 0) {
     prices <- prices %>%
      mutate(Symbol = symbol) %>%
      relocate(Symbol, .after = OpenTime)
    } else {
      print(glue("[bybit_prices] no prices for {symbol}"))
    }
    return (prices)
  }

  symbols %>%
    map_dfr(~fun(category, symbol = .x, interval, start, end)) %>%
    arrange(OpenTime)
}

In [ ]:
# symbols <- c("BTCUSDT", "SOLUSDT")
symbols <- c("BTCUSDT")
prices <- bybit_prices(
  # category = "spot",
  symbols = symbols,
  interval = "60",
  start = "2023-06-01",
  end = "2023-11-02"
)

prices

OpenTime,Symbol,Open,High,Low,Close,Volume,Turnover
<dttm>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2023-06-01 00:00:00,BTCUSDT,27211.51,27352.39,27027.19,27067.50,364.1892,9907528
2023-06-01 01:00:00,BTCUSDT,27067.50,27166.36,27062.74,27080.76,338.3078,9174435
2023-06-01 02:00:00,BTCUSDT,27080.76,27114.38,26650.45,26719.38,707.1064,18986263
2023-06-01 03:00:00,BTCUSDT,26719.38,26838.15,26609.31,26780.40,370.9620,9915351
2023-06-01 04:00:00,BTCUSDT,26780.40,26826.01,26756.93,26808.40,129.9128,3480179
2023-06-01 05:00:00,BTCUSDT,26808.40,26865.68,26790.00,26849.58,137.6029,3692906
2023-06-01 06:00:00,BTCUSDT,26849.58,26882.12,26778.70,26788.38,168.4887,4523289
2023-06-01 07:00:00,BTCUSDT,26788.38,26864.38,26766.00,26805.48,139.0436,3728814
2023-06-01 08:00:00,BTCUSDT,26805.48,26920.54,26791.23,26913.91,211.1701,5669813


# Funding Rate

https://bybit-exchange.github.io/docs/v5/market/history-fund-rate

-- Request Parameters (Parameter	Required	Type	Comments)
- category	true	string	Product type. linear,inverse
- symbol	true	string	Symbol name
- startTime	false	integer	The start timestamp (ms)
- endTime	false	integer	The end timestamp (ms)
- limit	false	integer	Limit for data size per page. [1, 200]. Default: 200

-- Response Parameters (Parameter	Type	Comments)
- category	string	Product type
- list	array	Object
- > symbol	string	Symbol name
- > fundingRate	string	Funding rate
- > fundingRateTimestamp	string	Funding rate timestamp (ms)

-- Request Example
- GET /v5/market/funding/history?category=linear&symbol=ETHPERP&limit=1 HTTP/1.1
- Host: api-testnet.bybit.com

```json
{
    "retCode": 0,
    "retMsg": "OK",
    "result": {
        "category": "linear",
        "list": [
            {
                "symbol": "ETHPERP",
                "fundingRate": "0.0001",
                "fundingRateTimestamp": "1672041600000"
            }
        ]
    },
    "retExtInfo": {},
    "time": 1672051897447
}
```


## single_bybit_funding_rates

In [ ]:
# Function to get Bybit funding rates
single_bybit_funding_rates <- function(category, symbol, start, end) {
  # Base URL for the Bybit API (funding history)
  base_url <- "https://api.bybit.com/v5/market/funding/history"

  start_unix = as.numeric(as.POSIXct(start, tz = "UTC", format = "%Y-%m-%d")) * 1000
  end_unix = as.numeric(as.POSIXct(glue("{end} 23:59:59"), tz = "UTC", format = "%Y-%m-%d %H:%M:%S")) * 1000

  # Create a list of parameters
  params <- list(category = category, symbol = symbol, limit = 200)
  params$startTime <- start_unix
  params$endTime <- end_unix

  all_data <- data.frame()

  repeat {
    response <- GET(url = base_url, query = params)

    if (response$status_code == 200) {
      data <- fromJSON(content(response, "text"), flatten = TRUE)

      if ("result" %in% names(data) && "list" %in% names(data$result)) {
        funding_data <- data$result$list

        if (length(funding_data) > 0) {
          funding_data <- as_tibble(funding_data) %>%
            mutate(
              Time = as.POSIXct(as.numeric(fundingRateTimestamp) / 1000, origin = "1970-01-01", tz = "UTC"),
              Symbol = symbol,
              Rate = as.numeric(fundingRate)
            ) %>%
            arrange(Time) %>%
            select(Time, Symbol, Rate)

          all_data <- rbind(all_data, funding_data)

          first_time <- min(funding_data$Time)
          end_unix <- as.numeric(first_time) * 1000 - 1
          params$endTime <- end_unix

          if (length(funding_data$Time) < params$limit || start_unix > end_unix) {
            break
          }
        } else {
          break
        }
      } else {
        warning("Unexpected data structure in the response.")
        return(all_data)
      }
    } else {
      warning("Request failed. Status: ", http_status(response)$status_code, " - ", http_status(response)$reason)
      return(all_data)
    }
  }

  if (length(all_data) > 0) {
    all_data <- all_data %>%
    arrange(Time)
  }

  return(all_data)
}

# Example usage
result <- single_bybit_funding_rates(category = "linear", start="2022-11-01", end="2023-11-02", symbol = "BTCUSDT")
print(result)

# A tibble: 1,101 × 3
   Time                Symbol         Rate
   <dttm>              <chr>         <dbl>
 1 2022-11-01 00:00:00 BTCUSDT  0.00000061
 2 2022-11-01 08:00:00 BTCUSDT -0.0000711 
 3 2022-11-01 16:00:00 BTCUSDT  0.0000305 
 4 2022-11-02 00:00:00 BTCUSDT  0.0000631 
 5 2022-11-02 08:00:00 BTCUSDT  0.0000241 
 6 2022-11-02 16:00:00 BTCUSDT  0.0000566 
 7 2022-11-03 00:00:00 BTCUSDT  0.0000734 
 8 2022-11-03 08:00:00 BTCUSDT -0.0000203 
 9 2022-11-03 16:00:00 BTCUSDT  0.0000650 
10 2022-11-04 00:00:00 BTCUSDT  0.000044  
# ℹ 1,091 more rows


## bybit_funding_rates - get multiple symbols funding history

In [ ]:
bybit_funding_rates <- function(category = "linear", symbols, start, end) {
  fun <- function(category, symbol, start, end) {
    print(glue("Loading Symbol {symbol}"))
    rates <- single_bybit_funding_rates(category, symbol, start, end)
    if (length(rates) == 0) {
      print(glue("[bybit_funding_rates] no rates for {symbol}"))
    }
    return(rates)
  }

  symbols %>%
    map_dfr(~fun(category, symbol = .x, start, end)) %>%
    arrange(Time)
}

symbols <- c("BTCUSDT", "SOLUSDT")
# symbols <- c("BTCUSDT")
rates <- bybit_funding_rates(
  # category = "linear",
  symbols = symbols,
  start = "2023-11-01",
  end = "2023-11-02"
)

rates

Loading Symbol BTCUSDT
Loading Symbol SOLUSDT


Time,Symbol,Rate
<dttm>,<chr>,<dbl>
2023-11-01 00:00:00,BTCUSDT,0.00010000
2023-11-01 00:00:00,SOLUSDT,0.00010000
2023-11-01 08:00:00,BTCUSDT,0.00011930
2023-11-01 08:00:00,SOLUSDT,0.00003923
2023-11-01 16:00:00,BTCUSDT,0.00010000
2023-11-01 16:00:00,SOLUSDT,-0.00028010
2023-11-02 00:00:00,BTCUSDT,0.00014309
2023-11-02 00:00:00,SOLUSDT,0.00010000
2023-11-02 08:00:00,BTCUSDT,0.00018541


# Account

- USDT Perpetual Trade History https://www.bybit.com/user/assets/order/derivatives/uniform-usdt/trade-history
- Spot Trade History https://www.bybit.com/user/assets/order/fed/spot-uta-orders/trade-order/current-order



In [ ]:
library(httr)
library(jsonlite)
library(dplyr)
library(lubridate)
library(openssl)
library(digest)

api_key <- "TuaZwcJvnvF2DUzVgo"
api_secret <- "btWyl1j3kxgqf291IOJyeyH5QYnzdeTywbtV"

# Function to concatenate parameters into a query string
concatenate_params <- function(params) {
  params_string <- sapply(names(params), function(key) paste0(key, "=", params[[key]]), USE.NAMES = FALSE)
  paste(params_string, collapse = "&")
}

# Function to generate signature
generate_signature <- function(api_key, secret_key, recv_window, query_string) {
  timestamp <- as.character(floor(as.numeric(Sys.time()) * 1000))
  param_str <- paste0(timestamp, api_key, recv_window, query_string)
  hmac(key = secret_key, object = param_str, algo = "sha256", serialize = FALSE, raw = FALSE)
}

## get_bybit_transaction_logs

- KAS/USDT Buy FilledValue:9.9989406USDT FilledPrice:0.13407USDT FilledQty:74.58KAS TradingFee:0.07458 KAS
- KASUSDT Short Filled:70/70 FilledPrice:0.13392/Market FeeRate:0.055% TradingFee:0.00515592

https://bybit-exchange.github.io/docs/v5/account/transaction-log

In [ ]:
# Function to get Bybit transaction logs with authentication
get_bybit_transaction_logs <- function(
    category="", currency="", baseCoin="", type="", start="", end="", limit = 50, cursor = "") {
  base_url <- "https://api.bybit.com/v5/account/transaction-log"
  recv_window <- "5000"
  # Prepare query string
  params <- list(accountType = "UNIFIED", category = category, currency = currency,
                 baseCoin = baseCoin, type = type, limit = limit, cursor = cursor)
  if (start != "") {
    params$startTime <- as.numeric(as.POSIXct(start, tz = "UTC", format = "%Y-%m-%d")) * 1000
  }
  if (end != "") {
    params$endTime <- as.numeric(as.POSIXct(end, tz = "UTC", format = "%Y-%m-%d")) * 1000
  }
  # Using `keep` to remove empty strings
  params <- keep(params, ~ nzchar(.x))
  query_string <- concatenate_params(params)
  # Generate current timestamp in milliseconds as a character string
  timestamp <- as.character(floor(as.numeric(Sys.time()) * 1000))
  # Generate signature
  signature <- generate_signature(api_key, api_secret, recv_window, query_string)
  # Create headers ensuring all values are character strings
  headers <- add_headers(`X-BAPI-API-KEY` = as.character(api_key),
                         `X-BAPI-SIGN` = as.character(signature),
                         `X-BAPI-TIMESTAMP` = as.character(timestamp),
                         `X-BAPI-RECV-WINDOW` = as.character(recv_window),
                         `Content-Type` = 'application/json')
    response <- GET(url = base_url, query = query_string, config = headers)
    # print(response)
    if (response$status_code == 200) {
      # Extract the response body
      response_content <- content(response, "text")
      # Parse the JSON content
      parsed_json <- fromJSON(response_content)
      # Print the result message
      print(parsed_json$retMsg)
      transactions_tibble <- as_tibble(parsed_json$result$list)
      # Check if 'transactionTime' column exists and tibble is not empty
      if ("transactionTime" %in% names(transactions_tibble) && nrow(transactions_tibble) > 0) {
        transactions_tibble <- transactions_tibble %>%
        mutate(transactionTime = as.POSIXct(as.numeric(transactionTime) / 1000, origin = "1970-01-01", tz = "UTC"))
      } else {
        print("got empty result")
      }
      return(transactions_tibble)
    } else {
      print("Error getting bybit response.")
    }
}

In [ ]:
# Example usage
result <- get_bybit_transaction_logs(
  # category = "linear",
  start = "2023-11-17",
  end = "2023-11-21"
) %>%
  select(-orderLinkId, -orderId, -bonusChange, -id, -tradeId)
# print(result)
result

[1] "OK"


symbol,side,funding,fee,change,cashFlow,transactionTime,type,feeRate,size,qty,cashBalance,currency,category,tradePrice
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dttm>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
KASUSDT,Sell,0.00091154,0,0.00091154,0,2023-11-21 00:00:00,SETTLEMENT,-0.0001,-70,70,149.04072601,USDT,linear,0.13022
KASUSDT,Sell,0.0055335,0,0.0055335,0,2023-11-20 16:00:00,SETTLEMENT,-0.000576,-70,70,149.03981447,USDT,linear,0.13744
KASUSDT,Sell,0.00470558,0,0.00470558,0,2023-11-20 08:00:00,SETTLEMENT,-0.000478,-70,70,149.03428097,USDT,linear,0.14068
KASUSDT,Sell,0.0047615,0,0.0047615,0,2023-11-20 00:00:00,SETTLEMENT,-0.000474,-70,70,149.02957539,USDT,linear,0.14369
KASUSDT,Sell,0.00100884,0,0.00100884,0,2023-11-19 16:00:00,SETTLEMENT,-0.0001,-70,70,149.02481389,USDT,linear,0.14412
KASUSDT,Sell,0.00100212,0,0.00100212,0,2023-11-19 08:00:00,SETTLEMENT,-0.0001,-70,70,149.02380505,USDT,linear,0.14316
KASUSDT,Sell,0.00145813,0,0.00145813,0,2023-11-19 00:00:00,SETTLEMENT,-0.000149,-70,70,149.02280293,USDT,linear,0.14013
KASUSDT,Sell,0.0009436,0,0.0009436,0,2023-11-18 16:00:00,SETTLEMENT,-0.0001,-70,70,149.0213448,USDT,linear,0.1348
KASUSDT,Sell,0,0.00515592,-0.00515592,0,2023-11-18 11:46:03,TRADE,0.00055,-70,70,149.0204012,USDT,linear,0.13392


## get_bybit_positions

In [ ]:
# Function to get Bybit transaction logs with authentication
get_bybit_positions <- function(
    category, symbol="", baseCoin="", settleCoin="", limit = 20, cursor = "") {
  base_url <- "https://api.bybit.com/v5/position/list"
  recv_window <- "5000"

  # Prepare query string
  params <- list(category = category, symbol = symbol,
                 baseCoin = baseCoin, settleCoin = settleCoin,
                 limit = limit, cursor = cursor)
  # Using `keep` to remove empty strings
  params <- keep(params, ~ nzchar(.x))
  query_string <- concatenate_params(params)

  # Generate current timestamp in milliseconds as a character string
  timestamp <- as.character(floor(as.numeric(Sys.time()) * 1000))
  # Generate signature
  signature <- generate_signature(api_key, api_secret, recv_window, query_string)

  # Create headers ensuring all values are character strings
  headers <- add_headers(`X-BAPI-API-KEY` = as.character(api_key),
                         `X-BAPI-SIGN` = as.character(signature),
                         `X-BAPI-TIMESTAMP` = as.character(timestamp),
                         `X-BAPI-RECV-WINDOW` = as.character(recv_window),
                         `Content-Type` = 'application/json')

    response <- GET(url = base_url, query = query_string, config = headers)

    if (response$status_code == 200) {
      # Assuming 'response' is the object you received from the GET request
      response_content <- content(response, "text")
      # Parse the JSON content
      parsed_json <- fromJSON(response_content)
      print(parsed_json$retMsg)
      # Accessing specific parts of the parsed JSON
      # For example, if you want to convert the 'list' inside 'result' to a data frame
      # transactions <- as.data.frame(parsed_json$result$list)
      result <- as_tibble(parsed_json$result$list)
      # transactions_tibble <- transactions_tibble %>%
      # mutate(transactionTime = as.POSIXct(as.numeric(transactionTime) / 1000, origin = "1970-01-01", tz = "UTC"))
      result
    } else {
      print("Error getting bybit response.")
    }
}

# Example usage
result <- get_bybit_positions(
  category = "linear",
  settleCoin = "USDT"
)
# print(result)
result

[1] "OK"


<0 x 0 matrix>

## get_bybit_order_history

https://bybit-exchange.github.io/docs/v5/order/order-list

In [ ]:
get_bybit_order_history <- function(
    category, symbol="", baseCoin="", settleCoin="",
    orderId="", orderLinkId="", orderFilter="", orderStatus="",
    startTime="", endTime="",
    limit = 20, cursor = "") {
  base_url <- "https://api.bybit.com/v5/order/history"
  if (startTime != "") {
    start_unix <- as.numeric(as.POSIXct(startTime, tz = "UTC", format = "%Y-%m-%d")) * 1000
    end_unix <- as.numeric(as.POSIXct(endTime, tz = "UTC", format = "%Y-%m-%d")) * 1000
  }

  recv_window <- "5000"

  # Prepare query string
  params <- list(category = category, symbol = symbol,
                 baseCoin = baseCoin, settleCoin = settleCoin,
                 orderId=orderId, orderLinkId=orderLinkId,
                 orderFilter=orderFilter, orderStatus=orderStatus,
                 startTime=startTime, endTime=endTime,
                 limit = limit, cursor = cursor)
  # Using `keep` to remove empty strings
  params <- keep(params, ~ nzchar(.x))
  query_string <- concatenate_params(params)

  # Generate current timestamp in milliseconds as a character string
  timestamp <- as.character(floor(as.numeric(Sys.time()) * 1000))
  # Generate signature
  signature <- generate_signature(api_key, api_secret, recv_window, query_string)

  # Create headers ensuring all values are character strings
  headers <- add_headers(`X-BAPI-API-KEY` = as.character(api_key),
                         `X-BAPI-SIGN` = as.character(signature),
                         `X-BAPI-TIMESTAMP` = as.character(timestamp),
                         `X-BAPI-RECV-WINDOW` = as.character(recv_window),
                         `Content-Type` = 'application/json')

    response <- GET(url = base_url, query = query_string, config = headers)

    if (response$status_code == 200) {
      # Assuming 'response' is the object you received from the GET request
      response_content <- content(response, "text")
      # Parse the JSON content
      parsed_json <- fromJSON(response_content)
      print(parsed_json$retMsg)
      # Accessing specific parts of the parsed JSON
      # For example, if you want to convert the 'list' inside 'result' to a data frame
      # transactions <- as.data.frame(parsed_json$result$list)
      result <- as_tibble(parsed_json$result$list)
      # transactions_tibble <- transactions_tibble %>%
      # mutate(transactionTime = as.POSIXct(as.numeric(transactionTime) / 1000, origin = "1970-01-01", tz = "UTC"))
      result
    } else {
      print("Error getting bybit response.")
    }
}

# Example usage
result <- get_bybit_order_history(
  category = "linear",
  # settleCoin = "USDT"
)
# print(result)
result

[1] "OK"


symbol,orderType,orderLinkId,slLimitPrice,orderId,cancelType,avgPrice,stopOrderType,lastPriceOnCreated,orderStatus,⋯,slTriggerBy,leavesQty,closeOnTrigger,placeType,cumExecQty,reduceOnly,qty,stopLoss,smpOrderId,triggerBy
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<lgl>,<chr>,<chr>,<lgl>,<chr>,<chr>,<chr>,<chr>
BTCUSDT,Market,,0,e5efb80b-e1bd-4395-b084-b88d2a9358dd,UNKNOWN,70071,,70071.1,Filled,⋯,,0,TRUE,,0.001,TRUE,0.001,,,
BTCUSDT,Limit,,0,cee270e3-cd4a-4002-8ad4-1ba305e4b3bf,UNKNOWN,70494.1,,70808,Filled,⋯,,0,FALSE,,0.001,FALSE,0.001,,,
BTCUSDT,Market,,0,40a352f3-f0ed-4aca-8caf-90f7294f7ba7,UNKNOWN,71848.8,,71850,Filled,⋯,,0,TRUE,,0.001,TRUE,0.001,,,
BTCUSDT,Market,,0,69da2ed3-b41a-42e2-9060-d875345558fa,UNKNOWN,71842.1,,71842.1,Filled,⋯,,0,FALSE,,0.001,FALSE,0.001,,,
BTCUSDT,Market,,0,a8aec6e2-6de5-4edc-b3e9-cc483a764b1e,UNKNOWN,71927,,71927.1,Filled,⋯,,0,TRUE,,0.001,TRUE,0.001,,,
BTCUSDT,Limit,,0,a89fc5a7-a065-4741-aa24-641fc18e0d80,UNKNOWN,71945,,71945,Filled,⋯,,0,FALSE,,0.001,FALSE,0.001,,,
BTCUSDT,Market,,0,c57454dd-4527-4a52-a696-387b6a6f58a4,UNKNOWN,72140,,72140.1,Filled,⋯,,0,TRUE,,0.001,TRUE,0.001,,,
BTCUSDT,Limit,,0,076534fb-329a-4b2d-997b-3a909abcd9dd,UNKNOWN,72166.1,,72166,Filled,⋯,,0,FALSE,,0.001,FALSE,0.001,,,


## get_bybit_balance

In [ ]:
get_bybit_balance <- function(
    memberId="", accountType="UNIFIED", coin="", withBonus="") {
  base_url <- "https://api.bybit.com/v5/asset/transfer/query-account-coins-balance"
  # if (startTime != "") {
  #   start_unix <- as.numeric(as.POSIXct(startTime, tz = "UTC", format = "%Y-%m-%d")) * 1000
  #   end_unix <- as.numeric(as.POSIXct(endTime, tz = "UTC", format = "%Y-%m-%d")) * 1000
  # }

  recv_window <- "5000"

  # Prepare query string
  params <- list(memberId = memberId, accountType = accountType,
                 coin = coin, withBonus = withBonus)
  # Using `keep` to remove empty strings
  params <- keep(params, ~ nzchar(.x))
  query_string <- concatenate_params(params)

  # Generate current timestamp in milliseconds as a character string
  timestamp <- as.character(floor(as.numeric(Sys.time()) * 1000))
  # Generate signature
  signature <- generate_signature(api_key, api_secret, recv_window, query_string)

  # Create headers ensuring all values are character strings
  headers <- add_headers(`X-BAPI-API-KEY` = as.character(api_key),
                         `X-BAPI-SIGN` = as.character(signature),
                         `X-BAPI-TIMESTAMP` = as.character(timestamp),
                         `X-BAPI-RECV-WINDOW` = as.character(recv_window),
                         `Content-Type` = 'application/json')

    response <- GET(url = base_url, query = query_string, config = headers)

    if (response$status_code == 200) {
      # Assuming 'response' is the object you received from the GET request
      response_content <- content(response, "text")
      # Parse the JSON content
      parsed_json <- fromJSON(response_content)
      print(parsed_json$retMsg)
      # Accessing specific parts of the parsed JSON
      # For example, if you want to convert the 'list' inside 'result' to a data frame
      # transactions <- as.data.frame(parsed_json$result$list)
      result <- as_tibble(parsed_json$result$list)
      # transactions_tibble <- transactions_tibble %>%
      # mutate(transactionTime = as.POSIXct(as.numeric(transactionTime) / 1000, origin = "1970-01-01", tz = "UTC"))
      result
    } else {
      print("Error getting bybit response.")
    }
}

# Example usage
result <- get_bybit_balance(
  # accountType = "CONTRACT"
)
# print(result)
result

[1] "Permission denied, please check your API key permissions."


<0 x 0 matrix>